In [5]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader



In [6]:
class LSTM(L.LightningModule):
    def __init__(self):
        super().__init__()
        # Init W using Normal Distribution
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)

        # Weights biases
        self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

        self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

    
    def lstmUnit(self, inputVal, longMem, shrtMem): #long/Short Memory  
        longRemeberPrecnt = torch.sigmoid((shrtMem * self.wlr1 + (inputVal * self.wlr2) + self.blr1))

        potentialRememberPrecnt = torch.sigmoid((shrtMem * self.wpr1) + (inputVal * self.wpr2) + self.bpr1)

        potentialMem = torch.tanh((shrtMem * self.wp1) + (inputVal * self.wp2) + self.bp1)

        updateLongTermMem = ((longMem * longRemeberPrecnt) + potentialRememberPrecnt * potentialMem)

        outputPrecnt = torch.sigmoid((shrtMem * self.wo1) + (inputVal * self.wo2) + self.bo1)   

        updateShortMem = torch.tanh(updateLongTermMem * outputPrecnt)

        return ([updateLongTermMem, updateShortMem])
    
    def forward(self, input):
        longMem = 0
        shortMem = 0

        # stock market data
        stockData = {
            'day1':input[0],
            'day2':input[1],
            'day3':input[2],
            'day4':input[3],
        }
        for day in stockData.values():
            longMem, shortMem = self.lstmUnit(day, longMem, shortMem)
        
        return shortMem
    
    def configOptim(self):
        return Adam(self.parameters())
    
    def trainStep(self, batch, batchIx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i) ** 2 #SSR

        self.log('Train loss', loss)

        if (label_i == 0):
            self.log('out_0', output_i)
        else:
            self.log('out_1', output_i)

        return loss
    

In [7]:
model =  LSTM()
print('observed and predict values')
print('Company A: Observed=0 predict =', model(torch.tensor([0., 0.5, 0.25, 1.])).detach())

print('Company B: Observed=1 predict =', model(torch.tensor([1., 0.5, 0.25, 1.])).detach())



observed and predict values
Company A: Observed=0 predict = tensor(-0.1869)
Company B: Observed=1 predict = tensor(-0.1887)


In [8]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataLoader = DataLoader(dataset, batch_size=2)

model = LSTM()
trainer = L.Trainer(
    max_epochs=2000, 
    accelerator='auto',
    devices='auto'
)
trainer.fit(model, train_dataloaders=dataLoader)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.